In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch
from torch.utils.data import  DataLoader,random_split
from torchvision import datasets,transforms
import torch.nn as nn

import sys
sys.path.append('/content/drive/MyDrive/mask-classfication')

In [4]:
from CustomModel import MaskNet , MaskNetV2
from MaskDataset import MaskImageDataset
import numpy as np
import time

import torch
from torchvision import transforms
from torch.utils.data import DataLoader, random_split, SubsetRandomSampler

from sklearn.model_selection import KFold
import matplotlib.pyplot as plt

In [5]:
data_path = "/content/drive/MyDrive/mask-classfication/dataset.csv"
img_path = "/content/drive/MyDrive/mask-classfication/Dataset"

from MaskDataset import MaskImageDataset


transform = transforms.Compose([transforms.Resize((224,224)),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                                ])

full_dataset = MaskImageDataset(data_path, img_path, transform=transform)

In [7]:
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
from torch.nn.functional import cross_entropy

from CustomModel import MaskNetV2

In [6]:
def trainModelWithKfold(model, train_loader, device, fold, test_loader):
    learning_rate = 0.001
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(),lr = learning_rate)
    
    train_losses = []
    acc_list = []
    epochs = 30
    
    for i in range(1, epochs+1):
        start = time.time()
        
        running_loss = 0.0
        total = 0
        correct = 0
        
        for j, data in enumerate(train_loader, 0):
            
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            
            total += labels.size(0)
            _, predicted = torch.max(outputs.data, 1) 
            correct += (predicted == labels).sum().item() 
        
            
        
        train_loss = running_loss/len(train_loader.sampler)
        train_losses.append(train_loss)
        accuracy = (correct / total) * 100
        acc_list.append(accuracy)
        
        print('Epoch: {} \tTraining Loss: {:.6f} \tAccuracy: {:.6f}'.format(
        i, train_loss, accuracy))
        elapsed = time.time() - start
        print("Elapsed time: " + time.strftime("%H:%M:%S.{}".format(str(elapsed % 1)[2:])[:11], time.gmtime(elapsed)))
    
    print(f"Finished Training of fold {fold}")
    #torch.save(model.state_dict(), f"G:/AI Project/mask-classification-pytorch/saved_models/masknetv2_k_fold{fold}_dict.pt")
    torch.save(model, F"/content/drive/MyDrive/mask-classfication/saved_models/masknetv2_k_fold::3{fold}_full.pt")

    testing_loss = 0
    correct_prediction = 0 
    data_size = 0
    prediction1=[]
    with torch.no_grad():
        
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)          
            data_size += len(images)
            prediction = model(images)
            
            prediction1.append(prediction)
            
            testing_loss += cross_entropy(prediction, labels).item()
            correct_prediction += (prediction.argmax(dim=1) == labels).sum().item()


    test_accuracy = correct_prediction/data_size
    testing_loss = testing_loss/data_size

    print('\nTesting:')
    print(f"Correct prediction: {correct_prediction}/{data_size} and accuracy: {test_accuracy} and loss: {testing_loss}")
    
    return (accuracy, test_accuracy)

In [8]:
def kFoldTraining(numFold, full_dataset):
    kfold = KFold(n_splits=numFold, shuffle=True)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    accuracies = []
    
    for fold, (train_ids, test_ids) in enumerate(kfold.split(full_dataset)):
        
        print(f"Fold {fold}")
        
        trainSubSampler = SubsetRandomSampler(train_ids)
        testSubSampler = SubsetRandomSampler(test_ids)
        
        trainloader = DataLoader(full_dataset, batch_size=32, sampler=trainSubSampler)
        testloader = DataLoader(full_dataset, batch_size=32, sampler=testSubSampler)
        
        model = MaskNetV2()
        model.apply(resetWeight)
        model = model.to(device)
        
        
        train_accuracy, test_accuracy = trainModelWithKfold(model, trainloader, device, fold, testloader)
        
        accuracies.append((train_accuracy, test_accuracy))
        
    
    return accuracies


In [10]:

def resetWeight(model):
    
    for layer in model.children():
        if hasattr(layer, 'reset_parameters'):
            layer.reset_parameters()
            
k_fold_accuracies = kFoldTraining(10, full_dataset)

Fold 0


/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


Epoch: 1 	Training Loss: 0.095876 	Accuracy: 29.445015
Elapsed time: 00:16:03.20
Epoch: 2 	Training Loss: 0.044113 	Accuracy: 41.007194
Elapsed time: 00:00:53.51
Epoch: 3 	Training Loss: 0.038101 	Accuracy: 46.865365
Elapsed time: 00:00:53.13
Epoch: 4 	Training Loss: 0.036322 	Accuracy: 51.130524
Elapsed time: 00:00:53.62
Epoch: 5 	Training Loss: 0.034705 	Accuracy: 53.648510
Elapsed time: 00:00:53.25
Epoch: 6 	Training Loss: 0.032766 	Accuracy: 56.526208
Elapsed time: 00:00:53.65
Epoch: 7 	Training Loss: 0.030376 	Accuracy: 59.558068
Elapsed time: 00:00:53.06
Epoch: 8 	Training Loss: 0.030260 	Accuracy: 60.123330
Elapsed time: 00:00:53.16
Epoch: 9 	Training Loss: 0.027772 	Accuracy: 64.182939
Elapsed time: 00:00:53.25
Epoch: 10 	Training Loss: 0.026775 	Accuracy: 65.107914
Elapsed time: 00:00:53.18
Epoch: 11 	Training Loss: 0.024735 	Accuracy: 68.139774
Elapsed time: 00:00:53.19
Epoch: 12 	Training Loss: 0.023403 	Accuracy: 70.554985
Elapsed time: 00:00:53.39
Epoch: 13 	Training Loss: